# Week 7 — Binary Search Trees & Heaps: Practice Notebook

This notebook consolidates the Week 7 material on **Binary Search Trees (BSTs)** and **Heaps** into clean, runnable Python, plus hands-on activities for students.

## Part I — Binary Search Trees (BST)

### 1) Node and BST class skeleton
We use a simple `Node` class and a `BST` class that tracks the `root_node`. The BST supports:
- `insert`
- `search`
- `find_min` / `find_max`
- `remove` (handles 0, 1, or 2 children)

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.left_child = None
        self.right_child = None

    def __repr__(self):
        return f"Node({self.data!r})"


class BST:
    def __init__(self):
        self.root_node = None

    def insert(self, data):
        node = Node(data)
        if self.root_node is None:
            self.root_node = node
            return node
        current = self.root_node
        while True:
            if data < current.data:
                if current.left_child is None:
                    current.left_child = node
                    return node
                current = current.left_child
            else:
                if current.right_child is None:
                    current.right_child = node
                    return node
                current = current.right_child

    def find_min(self):
        current = self.root_node
        if current is None:
            return None
        while current.left_child:
            current = current.left_child
        return current

    def find_max(self):
        current = self.root_node
        if current is None:
            return None
        while current.right_child:
            current = current.right_child
        return current

    def search(self, data):
        current = self.root_node
        while current:
            if data == current.data:
                return current
            elif data < current.data:
                current = current.left_child
            else:
                current = current.right_child
        return None

    def get_node_with_parent(self, data):
        parent = None
        current = self.root_node
        while current:
            if data == current.data:
                return parent, current
            parent = current
            if data < current.data:
                current = current.left_child
            else:
                current = current.right_child
        return None, None

    def remove(self, data):
        parent, node = self.get_node_with_parent(data)
        if node is None:
            return False
        children = 0
        if node.left_child:  children += 1
        if node.right_child: children += 1
        if children == 0:
            if parent is None:
                self.root_node = None
            else:
                if parent.left_child is node:
                    parent.left_child = None
                else:
                    parent.right_child = None
            return True
        if children == 1:
            next_node = node.left_child if node.left_child else node.right_child
            if parent is None:
                self.root_node = next_node
            else:
                if parent.left_child is node:
                    parent.left_child = next_node
                else:
                    parent.right_child = next_node
            return True
        parent_succ = node
        succ = node.right_child
        while succ.left_child:
            parent_succ = succ
            succ = succ.left_child
        node.data = succ.data
        if parent_succ.left_child is succ:
            parent_succ.left_child = succ.right_child
        else:
            parent_succ.right_child = succ.right_child
        return True


### 2) Traversals for validation
We include in-order, pre-order, and post-order to help you validate the BST content.

In [ ]:
def inorder(node):
    if node is None:
        return []
    return inorder(node.left_child) + [node.data] + inorder(node.right_child)

def preorder(node):
    if node is None:
        return []
    return [node.data] + preorder(node.left_child) + preorder(node.right_child)

def postorder(node):
    if node is None:
        return []
    return postorder(node.left_child) + postorder(node.right_child) + [node.data]


### 3) Quick demo: insert & search & remove

In [ ]:
bst = BST()
for x in [5, 3, 7, 1, 4, 6, 9, 8]:
    bst.insert(x)

print("In-order (sorted):", inorder(bst.root_node))
print("Min:", bst.find_min())
print("Max:", bst.find_max())
print("Search 6:", bst.search(6))

bst.remove(7)   # two children
print("In-order after removing 7:", inorder(bst.root_node))

bst.remove(1)   # leaf
print("In-order after removing 1:", inorder(bst.root_node))

bst.remove(5)   # root
print("In-order after removing 5:", inorder(bst.root_node))

### 4) Student Activities — BST
1. **Trace Insertions**: Insert `[5,3,7,1]` (and two more numbers). Draw the tree after each insertion. Verify with `inorder`.
2. **`kth_smallest(k)`**: Return the k-th smallest via in-order; target `O(h + k)` time.


## Part II — Heaps (Min-Heap)

### 5) Array-based min-heap
We implement a classic **min-heap** using a list where index `1` is the root.
- Parent: `i // 2`, Left: `2*i`, Right: `2*i + 1`.

In [ ]:
class MinHeap:
    def __init__(self):
        self.heap = [0]
        self.size = 0

    def _arrange_up(self, k):
        while k // 2 > 0:
            if self.heap[k] < self.heap[k // 2]:
                self.heap[k], self.heap[k // 2] = self.heap[k // 2], self.heap[k]
            k //= 2

    def insert(self, item):
        self.heap.append(item)
        self.size += 1
        self._arrange_up(self.size)

    def _minchild(self, k):
        if k * 2 + 1 > self.size:
            return k * 2
        left, right = k * 2, k * 2 + 1
        return left if self.heap[left] < self.heap[right] else right

    def _sink(self, k):
        while k * 2 <= self.size:
            mc = self._minchild(k)
            if self.heap[k] > self.heap[mc]:
                self.heap[k], self.heap[mc] = self.heap[mc], self.heap[k]
            k = mc

    def pop(self):
        if self.size == 0:
            raise IndexError("pop from empty heap")
        item = self.heap[1]
        self.heap[1] = self.heap[self.size]
        self.size -= 1
        self.heap.pop()
        if self.size > 0:
            self._sink(1)
        return item

    def peek(self):
        return None if self.size == 0 else self.heap[1]

    def build_heap(self, iterable):
        self.heap = [0] + list(iterable)
        self.size = len(self.heap) - 1
        i = self.size // 2
        while i > 0:
            self._sink(i)
            i -= 1


### 6) Quick demo: insert, pop, build_heap

In [ ]:
h = MinHeap()
for v in [3, 6, 2, 7, 5, 9, 4]:
    h.insert(v)

print("Heap array:", h.heap)
print("Peek (min):", h.peek())
print("Pop all:", [h.pop() for _ in range(h.size)])

h2 = MinHeap()
h2.build_heap([3, 6, 2, 7, 5, 9, 4])
sorted_values = []
while h2.size:
    sorted_values.append(h2.pop())
print("Heapsort result:", sorted_values)

### 7) Student Activities — Heaps
1. **Index Practice**: For node index `i`, compute parent/left/right. Verify on several `i`.
2. **Heapsort**: Implement `heapsort(iterable)` using `build_heap` and repeated `pop`.


## Part III — BST vs Heap: When to Use What

**BST**
- Ordered structure, in-order traversal yields sorted order; typical O(log n) ops if balanced.
- Can degrade to O(n) if unbalanced.

**Heap**
- Fast min/max access; priority queue; build in O(n); insert/pop in O(log n).
- No fast search for arbitrary keys; only root guaranteed min/max.

### 8) Activity — Choose the Structure
Pick BST or (min/max) heap for each scenario and defend your choice:
1. Continuously get the smallest deadline among tasks as they arrive.
2. Maintain a dynamic set and frequently check membership and iterate in sorted order.
3. Retrieve the k smallest items repeatedly as items stream in.
4. Merge k sorted lists efficiently (hint: priority queue).